Approach and Implementation

Step 1: Import state csv into the program by using pandas dataframes
If we are given 325 million synthetic individuals data:
age(19) x gender(2) x ethnicity(6) x income(16) x MSA(2) x state(51) = 372096 total groups

Step 2: Query LLM for Voting Probability
You can create a function, so that each time when you want to put something in LLM, you call this function

Step 3: randomly select one sample from each group/combination

Try the program by using variable_mapping.csv and AK.csv
combine step1,2,3 together, the program should be similar to this:

In [ ]:
import pandas as pd
import random

def get_prompt(random_sample, question, candidates):
    prompt = f"If the person's info is: {random_sample}\n"
    prompt += f"Here is the list of candidates:\n"
    for candidate in candidates:
        prompt += f"- {candidate.strip()}\n"

    prompt += (
    f"Based on this person's info, what is the probability of the question: {question}\n"
    "Please give your answer in the following format (each on a new line):\n"
    "Candidate1: xx%\n"
    "Candidate2: xx%\n"
    "Candidate3: xx%\n"
    "...\n"
    "Only give probabilities for each candidate in order.\n"
)
    print (prompt) # don't need to print, here is for explanation
    # Now this prompt is ready to be sent to the LLM
    return prompt


# a function that whenever import a new state you can use it.
def process_state(grouped, desc_cols, state_name):
    question = input("Enter your question: ")
    candidates = input("Enter ALL candidate names separated by commas: ").split(",")
    for group_key, group_df in grouped:
        # Randomly pick one row from this group (group_df)
        if not group_df.empty:
            random_row = group_df.sample(n=1).iloc[0]
            demographic_info = ', '.join([str(random_row[col]) for col in desc_cols])
            loc_desc = 'Small town' if random_row['loc_msa'] == 'S' else 'Large city'
            new_prompt = f"{demographic_info}, lives in {loc_desc}, in {state_name}"
            prompt = get_prompt(new_prompt, question, candidates) 
            # TODO: can call a new function called query_llm_api(prompt) to connect with LLM API

    return
        
if __name__ == '__main__':
    #Assume all data files in a folder called "data" next to this script
    # When run the program, use a for loop to access each data file in "data" folder
    df = pd.read_csv("data/AK.csv")
    vm = pd.read_csv("data/variable_mapping.csv")
    desc_cols = []

    for column in df.columns:
        vm_subset = vm[vm['var_id'] == column]

        if not vm_subset.empty:
            value_dict = dict(zip(vm_subset['value_id'], vm_subset['var_values']))
            description = vm_subset['description'].iloc[0].upper()
            df[description] = df[column].map(value_dict)
            desc_cols.append(description)
            
    group_keys = desc_cols + ['loc_msa']

    grouped = df.groupby(group_keys)

    process_state(grouped, desc_cols, 'Alaska')

we can pick one to see if this prompt can work and what does LLM return to this prompt (output hidden for brevity)
choose prompt: 

If the person's info is: 18 to 19 years, Female, Population of one race: American Indian and Alaska Native, $10 000 to $14 999, lives in Large city, in Alaska
Here is the list of candidates:
- Joe Biden
- Donald trump
- Bill Clinton
Based on this person's info, what is the probability that this person votes for each candidate?
Please give your answer in the following format (each on a new line):
Candidate1: xx%
Candidate2: xx%
Candidate3: xx%
...
Only give probabilities for each candidate in order.
----------------------------------------------------------------------------------------------------------

since I don't have authority to use LLM API in GitHub, I can't really apply the actual API here, but the outline is:
Create a new function (query_llm_api(prompt))
query_llm_api(prompt) is a function that send prompt to the chosen API, and store the result of the reply of API then convert into string

Sample string:
Joe Biden: 45%

Donald Trump: 50%

Bill Clinton: 5%

You call query_llm_api(prompt) inside of process_state(grouped, desc_cols, state_name) (This is where TODO)
then extract probability number in the returned string
------------------------------------------------------------------------------------------------------------

For example, this group has 3000 samples, and we get 3000 x 0.25 = 750 from LLM for voting for Biden, so 3000 x 0.65 = 1950 voting for Trump, and 3000 x 0.1 = 300 for Clinton
Store the total amount of ppl voting for each candidate.
You repeat those steps to get all groups in one state, then you move on to the next state.
Finally, compare total amount of voting for each candidate and draw the conclusion.


Metric:

To ensure the program works correctly, I checked (1) that prompts were generated for all groups, (2) all predicted probabilities were valid (0–1), (3) vote estimates were calculated as probability * group size, (4) total estimated votes were reasonable relative to population, and (5) sample prompts were reviewed for format accuracy.

Update 07/12

llm_voting_model.py is the file that accept other modules to import.
political_llm.py is the file that has a class which includes method to build prompt, connect LLM and output results returned by LLM

In this version, once the program starts, it automatically access state csv files inside of "data" folder. (I put AK.csv, VT.csv and WY.csv for testing). As I implement multiprocessing, all files may start process at the same time.

API connection is also availble here:
Key:BOtO6XUYDtqJ2hxmGjM18DN7AUI33nsJ8UAUsrtdh33vsvWIvFJtJQQJ99BGACHYHv6XJ3w3AAAAACOG0aWf
URL: https://jiahu-mcviy2ug-eastus2.cognitiveservices.azure.com/openai/deployments/gpt-35-turbo1/chat/completions?api-version=2025-01-01-preview
Deployment Name:gpt-35-turbo1

This version is able to connect to LLM and successfully get the result back.
The program is able to visualize the progress, the user can see it in terminal.



Testing:
See "Screenshot 2025-07-11 at 12.32.40 AM.png", "data" folder has 3 state csv files, and the program predict Joe Biden is the winner with around 650776 votes while there are 1,532,743 ppl in total when I input "Joe Biden, Bill Clinton, Donald Trump" as three candidates. This result seems plausible if there are 3 candidates. When here are 3 state files, it takes about 81 minutes, while 1 files takes about 40-50 minutes, using multiprocessing did make the program faster.